In [3]:
import difflib

# Investment options dataset
investment_options = [
    {
        "name": "Fixed Deposit",
        "risk": "low",
        "duration": ["short", "medium"],
        "liquidity": "medium",
        "tax_benefit": False,
        "goal_fit": ["capital_preservation", "retirement"],
        "user_type": ["conservative", "retiree"]
    },
    {
        "name": "Public Provident Fund (PPF)",
        "risk": "low",
        "duration": ["long"],
        "liquidity": "low",
        "tax_benefit": True,
        "goal_fit": ["retirement", "tax_saving"],
        "user_type": ["conservative"]
    },
    {
        "name": "Systematic Investment Plan (SIP - Equity MFs)",
        "risk": "medium",
        "duration": ["medium", "long"],
        "liquidity": "high",
        "tax_benefit": False,
        "goal_fit": ["wealth_creation"],
        "user_type": ["moderate", "aggressive"]
    },
    {
        "name": "Direct Equity (Stocks)",
        "risk": "high",
        "duration": ["medium", "long"],
        "liquidity": "high",
        "tax_benefit": False,
        "goal_fit": ["wealth_creation"],
        "user_type": ["aggressive", "moderate"]
    },
    {
        "name": "Health Insurance",
        "risk": "low",
        "duration": ["short", "long"],
        "liquidity": "not_applicable",
        "tax_benefit": True,
        "goal_fit": ["protection"],
        "user_type": ["everyone"]
    },
    {
        "name": "NPS",
        "risk": "medium",
        "duration": ["long"],
        "liquidity": "low",
        "tax_benefit": True,
        "goal_fit": ["retirement"],
        "user_type": ["moderate", "conservative"]
    },
    {
        "name": "Gold ETFs",
        "risk": "medium",
        "duration": ["medium", "long"],
        "liquidity": "high",
        "tax_benefit": False,
        "goal_fit": ["hedge"],
        "user_type": ["moderate", "aggressive"]
    },
    {
        "name": "Crypto Assets",
        "risk": "high",
        "duration": ["short", "medium"],
        "liquidity": "high",
        "tax_benefit": False,
        "goal_fit": ["wealth_creation"],
        "user_type": ["aggressive"]
    },
    {
        "name": "Real Estate",
        "risk": "medium",
        "duration": ["long"],
        "liquidity": "low",
        "tax_benefit": False,
        "goal_fit": ["wealth_creation", "capital_preservation"],
        "user_type": ["moderate", "aggressive"]
    },
    {
        "name": "Unit Linked Insurance Plan (ULIP)",
        "risk": "medium",
        "duration": ["long"],
        "liquidity": "low",
        "tax_benefit": True,
        "goal_fit": ["retirement", "tax_saving"],
        "user_type": ["moderate", "aggressive"]
    }
]

valid_risks = ["conservative", "moderate", "aggressive"]
valid_durations = ["short", "medium", "long"]
valid_goals = ["retirement", "wealth_creation", "capital_preservation", "tax_saving", "hedge", "protection"]

def map_existing_investments(user_input_list, investment_options):
    mapped_investments = []
    option_names = [inv["name"].lower() for inv in investment_options]

    abbreviation_map = {
        "sip": "systematic investment plan (sip - equity mfs)",
        "ppf": "public provident fund (ppf)",
        "fd": "fixed deposit",
        "nps": "nps",
        "ulip": "unit linked insurance plan (ulip)",
        "re": "real estate",
        "crypto": "crypto assets",
        "direct equity": "direct equity (stocks)",
        "health insurance": "health insurance"
    }

    for user_inv in user_input_list:
        user_inv_lower = user_inv.lower()
        if user_inv_lower in abbreviation_map:
            mapped_investments.append(abbreviation_map[user_inv_lower])
            continue

        matches = difflib.get_close_matches(user_inv_lower, option_names, n=1, cutoff=0.5)
        if matches:
            mapped_investments.append(matches[0])
        else:
            mapped_investments.append(user_inv_lower)

    return mapped_investments

def get_validated_input(prompt, valid_choices):
    while True:
        try:
            val = input(prompt).strip().lower()
            if val in valid_choices:
                return val
            else:
                print(f"Invalid choice! Please choose from {valid_choices}.")
        except Exception as e:
            print(f"Error: {e}")

def get_multiple_valid_inputs(prompt, valid_choices):
    while True:
        user_input = input(prompt).strip().lower()
        inputs = [x.strip() for x in user_input.split(",") if x.strip()]
        invalid = [x for x in inputs if x not in valid_choices]
        if invalid:
            print(f"Invalid choice(s): {invalid}. Please choose from {valid_choices}.")
        elif not inputs:
            print("You must provide at least one valid choice.")
        else:
            return inputs

def get_float_input(prompt):
    while True:
        try:
            val = input(prompt).strip()
            val_float = float(val)
            if val_float < 0:
                print("Value cannot be negative. Please enter a valid amount.")
                continue
            return val_float
        except ValueError:
            print("Invalid number. Please enter a valid numeric value.")

def get_user_input():
    print(" Please provide the following financial details:")

    age = int(get_float_input("Your Age: "))
    income = get_float_input("Monthly Income (INR): ")
    expenses = get_float_input("Monthly Expenses (INR): ")
    emi = get_float_input("Monthly EMI Payments (INR): ")
    debts = get_float_input("Total Personal Debts (INR): ")

    risk_appetite = get_validated_input(
        "Risk Appetite (conservative / moderate / aggressive): ", valid_risks
    )

    duration = get_validated_input(
        "Investment Duration (short / medium / long): ", valid_durations
    )

    goals = get_multiple_valid_inputs(
        "Enter your financial goals (comma-separated, e.g., retirement, tax_saving, capital_preservation,hedge,protection): ",
        valid_goals,
    )

    tax_benefit = get_validated_input(
        "Do you prefer tax-saving options? (yes/no): ", ["yes", "no"]
    ) == "yes"

    insurance_covered = get_validated_input(
        "Do you have sufficient insurance coverage? (yes/no): ", ["yes", "no"]
    ) == "yes"

    existing_input = input(
        "Enter existing investments (comma-separated, e.g., Fixed Deposit, PPF, SIP) or leave blank if none: "
    ).strip()
    if existing_input:
        raw_existing = [item.strip() for item in existing_input.split(",") if item.strip()]
        existing_portfolio = map_existing_investments(raw_existing, investment_options)
    else:
        existing_portfolio = []

    return {
        "age": age,
        "income": income,
        "expenses": expenses,
        "emi": emi,
        "personal_debt": debts,
        "risk_appetite": risk_appetite,
        "investment_duration": duration,
        "goals": goals,
        "needs_tax_benefit": tax_benefit,
        "insurance_coverage": insurance_covered,
        "existing_portfolio": existing_portfolio,
    }

def suggest_investments(user, options):
    recommendations = []
    restrictions = []
    warnings = []

    age = user["age"]
    annual_surplus = (user["income"] - user["expenses"] - user["emi"]) * 12
    emergency_required = user["expenses"] * 6

    # Age-based adjustment to risk appetite
    original_risk = user["risk_appetite"]
    if age < 30:
        user["risk_appetite"] = "aggressive"
    elif 30 <= age <= 45 and user["risk_appetite"] == "aggressive":
        user["risk_appetite"] = "moderate"
    elif age > 45:
        user["risk_appetite"] = "conservative"
    # Optionally notify about auto-adjustment
    if original_risk != user["risk_appetite"]:
        warnings.append(
            f"Based on your age ({age}), your risk profile was adjusted from '{original_risk}' to '{user['risk_appetite']}'."
        )

    # Emergency fund check
    if annual_surplus < emergency_required:
        recommendations.append("Emergency Fund (6 months of expenses)")
        warnings.append("Your savings may be insufficient for emergencies.")

    # Insurance check
    if not user["insurance_coverage"]:
        recommendations.append("Health Insurance")
        warnings.append("You currently lack insurance. Health Insurance is recommended.")

    for option in options:
        name_lower = option["name"].lower()

        # Restriction: Already invested
        if any(existing.lower() == name_lower for existing in user["existing_portfolio"]):
            restrictions.append(
                f"You have already invested in {option['name']}, so it is not repeated in your new recommendations. "
                "To maintain portfolio diversification and reduce risk concentration, consider exploring other asset types."
            )
            continue

        # Restriction: Risk profile mismatch
        if user["risk_appetite"] not in option["user_type"] and "everyone" not in option["user_type"]:
            allowed_profiles = [p.title() for p in option['user_type']]
            restrictions.append(
                f"{option['name']} is generally suited for {'/'.join(allowed_profiles)} investor profiles. "
                f"Since your current risk appetite is '{user['risk_appetite'].title()}', this option may expose you to an unsuitable level of risk."
            )
            continue

        # Restriction: Duration mismatch
        if user["investment_duration"] not in option["duration"]:
            restrictions.append(
                f"{option['name']} typically works best for {'/'.join(option['duration'])} investment durations. "
                f"You selected a '{user['investment_duration']}' duration, so this may not offer optimal returns or desired flexibility for your needs."
            )
            continue

        # Restriction: Goals mismatch
        if not any(goal in option["goal_fit"] for goal in user["goals"]):
            option_goals = [g.replace('_', ' ').title() for g in option["goal_fit"]]
            user_goals = [g.replace('_', ' ').title() for g in user["goals"]]
            restrictions.append(
                f"{option['name']} is primarily designed for goals like: {', '.join(option_goals)}. "
                f"This does not closely match your selected goal(s): {', '.join(user_goals)}. "
                "Selecting investments aligned with your objectives improves the likelihood of meeting them."
            )
            continue

        # Restriction: No tax benefit (if needed)
        if user["needs_tax_benefit"] and not option["tax_benefit"]:
            restrictions.append(
                f"{option['name']} does not offer tax-saving benefits. "
                "As you indicated a preference for tax-saving investments, this option is excluded to help you optimize your tax bill."
            )
            continue

        recommendations.append(option["name"])

    if not recommendations:
        warnings.append("No suitable investment options found. Please review your inputs or consider revising your goals, risk appetite, or investment duration.")

    # Budget Allocation Suggestion
    budget_plan = {
        "Emergency Fund": round(min(annual_surplus * 0.3, emergency_required), 2),
        "Insurance": 0 if user["insurance_coverage"] else round(annual_surplus * 0.15, 2),
        "Retirement": round(annual_surplus * 0.25 if age >= 35 else annual_surplus * 0.15, 2),
        "Tax-saving": round(annual_surplus * 0.2 if user["needs_tax_benefit"] else 0, 2),
        "Wealth Creation": round(annual_surplus * 0.4, 2),
    }

    return recommendations, restrictions, warnings, budget_plan

if __name__ == "__main__":
    user_profile = get_user_input()
    recs, restrs, warns, budget = suggest_investments(user_profile, investment_options)

    print("\nRecommended Investment Options for You:")
    for i, option in enumerate(recs, 1):
        print(f"{i}. {option}")

    print("\nRestrictions / Skipped Options (and reasons):")
    if restrs:
        for r in restrs:
            print(f"- {r}")
    else:
        print("None")

    print("\nWarnings / Suggestions:")
    if warns:
        for w in warns:
            print(f"- {w}")
    else:
        print("None")

    print("\n📊 Suggested Budget Allocation from Annual Surplus:")
    for category, amount in budget.items():
        print(f"{category}: ₹{amount:,.2f}")


 Please provide the following financial details:


Your Age:  30
Monthly Income (INR):  50000
Monthly Expenses (INR):  40
Monthly EMI Payments (INR):  45
Total Personal Debts (INR):  89
Risk Appetite (conservative / moderate / aggressive):  moderate
Investment Duration (short / medium / long):  long
Enter your financial goals (comma-separated, e.g., retirement, tax_saving, capital_preservation,hedge,protection):  retirement
Do you prefer tax-saving options? (yes/no):  no
Do you have sufficient insurance coverage? (yes/no):  yes
Enter existing investments (comma-separated, e.g., Fixed Deposit, PPF, SIP) or leave blank if none:  



Recommended Investment Options for You:
1. NPS
2. Unit Linked Insurance Plan (ULIP)

Restrictions / Skipped Options (and reasons):
- Fixed Deposit is generally suited for Conservative/Retiree investor profiles. Since your current risk appetite is 'Moderate', this option may expose you to an unsuitable level of risk.
- Public Provident Fund (PPF) is generally suited for Conservative investor profiles. Since your current risk appetite is 'Moderate', this option may expose you to an unsuitable level of risk.
- Systematic Investment Plan (SIP - Equity MFs) is primarily designed for goals like: Wealth Creation. This does not closely match your selected goal(s): Retirement. Selecting investments aligned with your objectives improves the likelihood of meeting them.
- Direct Equity (Stocks) is primarily designed for goals like: Wealth Creation. This does not closely match your selected goal(s): Retirement. Selecting investments aligned with your objectives improves the likelihood of meeting t

In [5]:
import difflib
import dash
from dash import dcc, html, Input, Output, State, dash_table
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px

# Investment options dataset
investment_options = [
    {
        "name": "Fixed Deposit",
        "risk": "low",
        "duration": ["short", "medium"],
        "liquidity": "medium",
        "tax_benefit": False,
        "goal_fit": ["capital_preservation", "retirement"],
        "user_type": ["conservative", "retiree"]
    },
    {
        "name": "Public Provident Fund (PPF)",
        "risk": "low",
        "duration": ["long"],
        "liquidity": "low",
        "tax_benefit": True,
        "goal_fit": ["retirement", "tax_saving"],
        "user_type": ["conservative"]
    },
    {
        "name": "Systematic Investment Plan (SIP - Equity MFs)",
        "risk": "medium",
        "duration": ["medium", "long"],
        "liquidity": "high",
        "tax_benefit": False,
        "goal_fit": ["wealth_creation"],
        "user_type": ["moderate", "aggressive"]
    },
    {
        "name": "Direct Equity (Stocks)",
        "risk": "high",
        "duration": ["medium", "long"],
        "liquidity": "high",
        "tax_benefit": False,
        "goal_fit": ["wealth_creation"],
        "user_type": ["aggressive", "moderate"]
    },
    {
        "name": "Health Insurance",
        "risk": "low",
        "duration": ["short", "long"],
        "liquidity": "not_applicable",
        "tax_benefit": True,
        "goal_fit": ["protection"],
        "user_type": ["everyone"]
    },
    {
        "name": "NPS",
        "risk": "medium",
        "duration": ["long"],
        "liquidity": "low",
        "tax_benefit": True,
        "goal_fit": ["retirement"],
        "user_type": ["moderate", "conservative"]
    },
    {
        "name": "Gold ETFs",
        "risk": "medium",
        "duration": ["medium", "long"],
        "liquidity": "high",
        "tax_benefit": False,
        "goal_fit": ["hedge"],
        "user_type": ["moderate", "aggressive"]
    },
    {
        "name": "Crypto Assets",
        "risk": "high",
        "duration": ["short", "medium"],
        "liquidity": "high",
        "tax_benefit": False,
        "goal_fit": ["wealth_creation"],
        "user_type": ["aggressive"]
    },
    {
        "name": "Real Estate",
        "risk": "medium",
        "duration": ["long"],
        "liquidity": "low",
        "tax_benefit": False,
        "goal_fit": ["wealth_creation", "capital_preservation"],
        "user_type": ["moderate", "aggressive"]
    },
    {
        "name": "Unit Linked Insurance Plan (ULIP)",
        "risk": "medium",
        "duration": ["long"],
        "liquidity": "low",
        "tax_benefit": True,
        "goal_fit": ["retirement", "tax_saving"],
        "user_type": ["moderate", "aggressive"]
    }
]

valid_risks = ["conservative", "moderate", "aggressive"]
valid_durations = ["short", "medium", "long"]
valid_goals = ["retirement", "wealth_creation", "capital_preservation", "tax_saving", "hedge", "protection"]

def map_existing_investments(user_input_list, investment_options):
    mapped_investments = []
    option_names = [inv["name"].lower() for inv in investment_options]

    abbreviation_map = {
        "sip": "systematic investment plan (sip - equity mfs)",
        "ppf": "public provident fund (ppf)",
        "fd": "fixed deposit",
        "nps": "nps",
        "ulip": "unit linked insurance plan (ulip)",
        "re": "real estate",
        "crypto": "crypto assets",
        "direct equity": "direct equity (stocks)",
        "health insurance": "health insurance"
    }

    for user_inv in user_input_list:
        user_inv_lower = user_inv.lower()
        if user_inv_lower in abbreviation_map:
            mapped_investments.append(abbreviation_map[user_inv_lower])
            continue

        matches = difflib.get_close_matches(user_inv_lower, option_names, n=1, cutoff=0.5)
        if matches:
            mapped_investments.append(matches[0])
        else:
            mapped_investments.append(user_inv_lower)
    return mapped_investments

def suggest_investments(user, options):
    recommendations = []
    restrictions = []
    warnings = []

    age = user["age"]
    annual_surplus = (user["income"] - user["expenses"] - user["emi"]) * 12
    emergency_required = user["expenses"] * 6

    original_risk = user["risk_appetite"]
    if age < 30:
        user["risk_appetite"] = "aggressive"
    elif 30 <= age <= 45 and user["risk_appetite"] == "aggressive":
        user["risk_appetite"] = "moderate"
    elif age > 45:
        user["risk_appetite"] = "conservative"
    if original_risk != user["risk_appetite"]:
        warnings.append(
            f"Based on your age ({age}), your risk profile was adjusted from '{original_risk}' to '{user['risk_appetite']}'."
        )

    if annual_surplus < emergency_required:
        recommendations.append("Emergency Fund (6 months of expenses)")
        warnings.append("Your savings may be insufficient for emergencies.")

    if not user["insurance_coverage"]:
        recommendations.append("Health Insurance")
        warnings.append("You currently lack insurance. Health Insurance is recommended.")

    for option in options:
        name_lower = option["name"].lower()
        if any(existing.lower() == name_lower for existing in user["existing_portfolio"]):
            restrictions.append(
                f"You have already invested in {option['name']}, so it is not repeated in your new recommendations. "
                "To maintain portfolio diversification and reduce risk concentration, consider exploring other asset types."
            )
            continue
        if user["risk_appetite"] not in option["user_type"] and "everyone" not in option["user_type"]:
            allowed_profiles = [p.title() for p in option['user_type']]
            restrictions.append(
                f"{option['name']} is generally suited for {'/'.join(allowed_profiles)} investor profiles. "
                f"Since your current risk appetite is '{user['risk_appetite'].title()}', this option may expose you to an unsuitable level of risk."
            )
            continue
        if user["investment_duration"] not in option["duration"]:
            restrictions.append(
                f"{option['name']} typically works best for {'/'.join(option['duration'])} investment durations. "
                f"You selected a '{user['investment_duration']}' duration, so this may not offer optimal returns or desired flexibility for your needs."
            )
            continue
        if not any(goal in option["goal_fit"] for goal in user["goals"]):
            option_goals = [g.replace('_', ' ').title() for g in option["goal_fit"]]
            user_goals = [g.replace('_', ' ').title() for g in user["goals"]]
            restrictions.append(
                f"{option['name']} is primarily designed for goals like: {', '.join(option_goals)}. "
                f"This does not closely match your selected goal(s): {', '.join(user_goals)}. "
                "Selecting investments aligned with your objectives improves the likelihood of meeting them."
            )
            continue
        if user["needs_tax_benefit"] and not option["tax_benefit"]:
            restrictions.append(
                f"{option['name']} does not offer tax-saving benefits. "
                "As you indicated a preference for tax-saving investments, this option is excluded to help you optimize your tax bill."
            )
            continue
        recommendations.append(option["name"])

    if not recommendations:
        warnings.append("No suitable investment options found. Please review your inputs or consider revising your goals, risk appetite, or investment duration.")

    budget_plan = {
        "Emergency Fund": round(min(annual_surplus * 0.3, emergency_required), 2),
        "Insurance": 0 if user["insurance_coverage"] else round(annual_surplus * 0.15, 2),
        "Retirement": round(annual_surplus * 0.25 if age >= 35 else annual_surplus * 0.15, 2),
        "Tax-saving": round(annual_surplus * 0.2 if user["needs_tax_benefit"] else 0, 2),
        "Wealth Creation": round(annual_surplus * 0.4, 2),
    }

    return recommendations, restrictions, warnings, budget_plan
# --------- Dash App Layout --------------

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MINTY])

def build_recommendations_card(title, items, color="primary"):
    if not items:
        return dbc.Alert("None", color="light", className="mb-2")
    return dbc.Card([
        dbc.CardHeader(title, className="bg-"+color+" text-white"),
        dbc.CardBody([
            html.Ul([html.Li(item) for item in items], style={'fontSize':'1.1em'})
        ])
    ], className="mb-4 shadow-sm")

def build_budget_pie(budget_plan):
    df = pd.DataFrame({
        "Category": list(budget_plan.keys()),
        "Amount": list(budget_plan.values())
    })
    # Filter out zero allocations for better chart clarity
    df = df[df["Amount"] > 0]
    fig = px.pie(df, names="Category", values="Amount", 
                 hole=0.4, 
                 color_discrete_sequence=px.colors.sequential.Tealgrn,
                 height=350)
    fig.update_traces(textinfo='percent+label')
    fig.update_layout(margin=dict(t=25, b=0, r=0, l=0))
    return dcc.Graph(figure=fig, config={'displayModeBar': False})

app.layout = dbc.Container([
    html.H1("Investment Recommendations Tool", className="mb-4 mt-3 text-center text-primary"),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Enter Your Financial Details", className="bg-primary text-white"),
                dbc.CardBody([
                    dbc.Row([
                        dbc.Col([
                            dbc.Label("Age"),
                            dbc.Input(type="number", id="age", min=0, max=120, step=1, value=30)
                        ], md=6),
                        dbc.Col([
                            dbc.Label("Monthly Income (INR)"),
                            dbc.Input(type="number", id="income", min=0, step=100, value=50000)
                        ], md=6),
                    ], className="mb-2"),
                    dbc.Row([
                        dbc.Col([
                            dbc.Label("Monthly Expenses (INR)"),
                            dbc.Input(type="number", id="expenses", min=0, step=100, value=25000)
                        ], md=6),
                        dbc.Col([
                            dbc.Label("Monthly EMI Payments (INR)"),
                            dbc.Input(type="number", id="emi", min=0, step=100, value=5000)
                        ], md=6)
                    ], className="mb-2"),
                    dbc.Row([
                        dbc.Col([
                            dbc.Label("Total Personal Debts (INR)"),
                            dbc.Input(type="number", id="debts", min=0, step=1000, value=0)
                        ], md=6),
                        dbc.Col([
                            dbc.Label("Risk Appetite"),
                            dcc.Dropdown(
                                id="risk",
                                options=[{"label": x.title(), "value": x} for x in valid_risks],
                                value="moderate", clearable=False),
                        ], md=6)
                    ], className="mb-2"),
                    dbc.Row([
                        dbc.Col([
                            dbc.Label("Investment Duration"),
                            dcc.Dropdown(
                                id="duration",
                                options=[{"label": x.title(), "value": x} for x in valid_durations],
                                value="medium", clearable=False),
                        ], md=6),
                        dbc.Col([
                            dbc.Label("Financial Goals"),
                            dcc.Dropdown(
                                id="goals",
                                options=[{"label": x.replace('_',' ').title(), "value": x} for x in valid_goals],
                                multi=True, value=["wealth_creation"]),
                        ], md=6)
                    ], className="mb-2"),
                    dbc.Row([
                        dbc.Col([
                            dbc.Label("Need Tax Saving Investment?"),
                            dcc.RadioItems(
                                id="tax_benefit",
                                options=[{"label":"Yes","value":True},{"label":"No","value":False}],
                                value=False,
                                inline=True)
                        ], md=6),
                        dbc.Col([
                            dbc.Label("Have Sufficient Insurance Coverage?"),
                            dcc.RadioItems(
                                id="insurance_covered",
                                options=[{"label":"Yes","value":True},{"label":"No","value":False}],
                                value=False,
                                inline=True)
                        ], md=6)
                    ], className="mb-2"),
                    dbc.Row([
                        dbc.Col([
                            dbc.Label("Existing Investments"),
                            dcc.Input(id="existing", type="text", placeholder="SIP, PPF, etc.", style={"width":"100%"}),
                            html.Div("Separate by commas, abbreviations allowed (ex: SIP, FD, PPF, NPS, ULIP)", style={"fontSize":"0.85em", "color":"gray"}),
                        ])
                    ], className="mb-2"),
                    # CHANGED line below:
                    dbc.Button("Get Recommendations", id="submit", color="success", className="mt-3 w-100"),
                ])
            ], className="mb-4 shadow-lg")
        ], md=6),

        dbc.Col([
            html.Div(id="results", children=[
                html.Div("Please enter your details and click 'Get Recommendations'!", className="text-muted text-center pt-5")
            ])
        ], md=6),
    ])
], fluid=True)

@app.callback(
    Output("results", "children"),
    Input("submit", "n_clicks"),
    [
        State("age", "value"),
        State("income", "value"),
        State("expenses", "value"),
        State("emi", "value"),
        State("debts", "value"),
        State("risk", "value"),
        State("duration", "value"),
        State("goals", "value"),
        State("tax_benefit", "value"),
        State("insurance_covered", "value"),
        State("existing", "value")
    ]
)
def update_recommendation(n_clicks, age, income, expenses, emi, debts, risk, duration, goals, tax_benefit, insurance_covered, existing):
    if not n_clicks:
        return dash.no_update

    existing_portfolio = []
    if existing:
        portfolio_list = [item.strip() for item in existing.split(",") if item.strip()]
        existing_portfolio = map_existing_investments(portfolio_list, investment_options)

    user_profile = {
        "age": int(age) if age else 30,
        "income": float(income) if income else 0,
        "expenses": float(expenses) if expenses else 0,
        "emi": float(emi) if emi else 0,
        "personal_debt": float(debts) if debts else 0,
        "risk_appetite": risk,
        "investment_duration": duration,
        "goals": goals or [],
        "needs_tax_benefit": tax_benefit,
        "insurance_coverage": insurance_covered,
        "existing_portfolio": existing_portfolio,
    }

    recs, restrs, warns, budget = suggest_investments(user_profile, investment_options)

    card_recommend = build_recommendations_card("Recommended Investment Options", recs, color="success")
    card_restrictions = build_recommendations_card("Restrictions / Skipped Options (Reasons)", restrs, color="warning")
    card_warnings = build_recommendations_card("Warnings / Suggestions", warns, color="danger")

    budget_row = [
        html.H5("📊 Suggested Budget Allocation from Annual Surplus", className="mt-4"),
        build_budget_pie(budget)
    ]

    return [
        card_recommend,
        card_restrictions,
        card_warnings,
        *budget_row
    ]

if __name__ == "__main__":
    app.run(debug=True, port=8090)
